<a href="https://colab.research.google.com/github/LennartKeller/TextklassifikationsProjekt2019/blob/master/HyperparamOptimization_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from typing import Union, List

import numpy as np
from scipy.sparse import csr_matrix
from sklearn.base import BaseEstimator
from sklearn.exceptions import NotFittedError
from sklearn.metrics import f1_score
from tqdm import tqdm


class PeriodEstimatorWrapper(BaseEstimator):

    def __init__(self, clf: BaseEstimator, **params):
        self.clf = clf(**params)
        if params.get('verbose'):
            self.verbose = params['verbose']

    def fit(self, X_train: Union[csr_matrix, np.ndarray], y_train: np.array):
        """
        Fits the estimator.

        :param X_train: normal feature matrix e.g. shape (n_samples, n_features)
        :param y_train: label vector shape (n_samples,)
        :return: fitted instance of itself
        """

        self.clf.fit(X_train, y_train)
        self.fitted_ = True

        return self

    def predict(self, X_test: List[Union[csr_matrix, np.ndarray]]):
        """
        Predicts classes for n periods
        :param X_test: list of feature matrices (n_samples, n_features) to predict (one for each period)
        :return: list of predicted label vectors
        """

        if not self.fitted_:
            raise NotFittedError

        result = []
        if self.verbose:
            iterator = tqdm(X_test, desc='Predicting classes for periods')
        else:
            iterator = X_test

        for X in iterator:
            result.append(self.clf.predict(X))

        return result

    def predict_proba(self, X_test: List[Union[csr_matrix, np.ndarray]]):
        """
        Predicts probabilities for n periods
        :param X_test: list of feature matrices (n_samples, n_features) to predict (one for each period)
        :return: list of predicted label vectors
        """
        if not hasattr(self.clf, 'predict_proba'):
            raise Exception(f"Method predict_proba is not implemented in {self.clf.__class__.__name__}")

        if not self.fitted_:
            raise NotFittedError

        result = []
        if self.verbose:
            iterator = tqdm(X_test, desc='Predicting classes for periods')
        else:
            iterator = X_test

        for X in iterator:
            result.append(self.clf.predict_proba(X))

        return result

    def decision_function(self, X_test: List[Union[csr_matrix, np.ndarray]]):
        """
        Predicts decision scores for n periods
        :param X_test: list of feature matrices (n_samples, n_features) to predict (one for each period)
        :return: list of predicted label vectors
        """
        if not hasattr(self.clf, 'decision_function'):
            raise Exception(f"Method decision_function is not implemented in {self.clf.__class__.__name__}")

        if not self.fitted_:
            raise NotFittedError

        result = []
        if self.verbose:
            iterator = tqdm(X_test, desc='Predicting classes for periods')
        else:
            iterator = X_test

        for X in iterator:
            result.append(self.clf.predict_proba(X))

        return result

    def score(self,
              X_test: List[Union[csr_matrix, np.ndarray]],
              y_true: List[np.array],
              scoring_func: callable = lambda y_true, y_pred: f1_score(y_true, y_pred, average='macro'),
              pooling_func: callable = np.mean):

        if not self.fitted_:
            raise NotFittedError

        scores = []
        for X, y in zip(X_test, y_true):
            y_pred = self.clf.predict(X)
            score = scoring_func(y, y_pred)
            scores.append(score)

        return pooling_func(scores)


### Problem: Wie tunen wir die Hyperparameter?

Problem: Unsere Idee sieht vor ein Modell auf alle Genres innerhalb einer "Periode" zu trainieren und auf alle anderen anzuwenden, um abzuschätzen wie sehr sich die Genres über die Zeit verändern. Hierbei stellt sich die Frage, wie man die Hyperparameter der Modelle valide und gleichzeitig effektiv optimieren kann.

* Möglichkeit 1:
    * Gridsearch auf Ausgangsperiode
    * Vorteile:
        * Wahrscheinlich am ehesten valide
    * Nachteile:
        * Unsere Datengrundlage ist zu klein, um dass für einzelne Epochen sinnvoll durchzuführen
* Möglichkeit 2:
    * Gridsearch auf allen Daten
    * Vorteile:
        * Große Datenmenge
        * Modell würde auf alle Eigenheiten der Perioden getuned werden (wobei das eher ein Nachteil ist)
    * Nachteil:
        * Spätere Testdaten würden fürs Optimieren verwendet werden
* Möglichkeit 3:
    * ParamDict verwenden, um die den eigentlich Lauf (das Trainieren auf einer Epoche und Testen auf allen Anderen) mit allen möglichen Hyperparamtern zu testen. Eigene Evaulation (bsp. Mittelwert der F1-Scores für die verschiedenen Epochen)
    * Vorteile:
        * Klare Trennung von Test und Trainingsdaten
        * Mehr Daten für die Optimierung als bei Möglichkeit 1
    * Nachteile:
        * keine cross-validation

In [0]:
import pandas as pd

In [3]:
!pip install stop_words

  Created wheel for stop-words: filename=stop_words-2018.7.23-cp36-none-any.whl size=32916 sha256=bf1c6765a76aaaf78c57a04a3d594ca29d5da39d0ed4b133e7b99941d4c9a9fc
  Stored in directory: /root/.cache/pip/wheels/75/37/6a/2b295e03bd07290f0da95c3adb9a74ba95fbc333aa8b0c7c78
Successfully built stop-words


In [4]:
from google.colab import drive

drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [5]:
!ls /content/gdrive/My\ Drive/

'Colab Notebooks'   full_dataset.csv	     smv_tuning1.csv
 data		    full_taggeddataset.csv


In [0]:
df = pd.read_csv('/content/gdrive/My Drive/full_taggeddataset.csv')

In [0]:
# remove news genre

df = df[df.genre != 'NEWS']
df = df[df.genre != 'NEWS-P4']
df.region = df.region.str.upper()
df = df[df.year != 'GesetzsammlungThÅringen']
df = df[df.year != 'GesetzsammlungThüringen']
df = df[df.year != '1851-54']

In [0]:
df_p1 = df.loc[df['period'] == 'P1']
df_rest = df.loc[df['period'] != 'P1']

# Feature Extraction

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
from stop_words import get_stop_words

tfidf = TfidfVectorizer()

# Bauen der Pipeline

In [0]:
from sklearn.pipeline import make_pipeline, make_union, Pipeline

In [0]:
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier

In [12]:
pipe_svm = Pipeline([('tfidf', tfidf), ('linearsvc', LinearSVC())])
pipe_svm

Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('linearsvc',
                 LinearSVC(C=1.0, class_weight=None, dual=True,
                           fit_intercept=True, intercept_scaling=1,
         

In [0]:
pipe_svm_params = {
    'tfidf__max_features': [300, 1000, 5000,  10000, 20000],
    'tfidf__ngram_range': [(1,1), (1,3), (1,5)],
    'tfidf_lowercase': [True, False],
    'tfidf_stopwords': [None, get_stop_words('de')],
    'linearsvc__C': [0.1, 0.5, 1, 3, 7, 10],
    'linearsvc__penalty': ['l2', 'l1']
    
}

# 1. Möglichkeit: Gridsearch auf Trainingsperiode

In [0]:
from sklearn.model_selection import GridSearchCV

gridsearch = GridSearchCV(
    pipe_svm,
    pipe_svm_params,
    scoring='f1_macro',
    verbose=1,
    n_jobs=-1)

In [0]:
gridsearch.fit(df_p1.text, df_p1.genre.to_numpy())

Fitting 5 folds for each of 900 candidates, totalling 4500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:  6.6min
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed: 12.8min
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed: 23.4min
[Parallel(n_jobs=-1)]: Done 1246 tasks      | elapsed: 36.4min
[Parallel(n_jobs=-1)]: Done 1796 tasks      | elapsed: 51.8min
[Parallel(n_jobs=-1)]: Done 2446 tasks      | elapsed: 71.9min
[Parallel(n_jobs=-1)]: Done 3196 tasks      | elapsed: 92.3min
[Parallel(n_jobs=-1)]: Done 4046 tasks      | elapsed: 117.0min
[Parallel(n_jobs=-1)]: Done 4500 out of 4500 | elapsed: 130.0min finished


GridSearchCV(cv=None, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('tfidf',
                                        TfidfVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.float64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                                      

In [0]:
gridsearch.best_params_, gridsearch.best_score_

({'linearsvc__C': 3,
  'linearsvc__penalty': 'l2',
  'tfidf__analyzer': 'word',
  'tfidf__max_features': 15000,
  'tfidf__ngram_range': (1, 1)},
 0.7830952380952381)

In [0]:
svm_results = pd.DataFrame.from_dict(gridsearch.cv_results_)
svm_results

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_linearsvc__C,param_linearsvc__penalty,param_tfidf__analyzer,param_tfidf__max_features,param_tfidf__ngram_range,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.322687,0.003073,0.052285,0.002864,1,l2,word,1000,"(1, 1)","{'linearsvc__C': 1, 'linearsvc__penalty': 'l2'...",0.708730,0.659524,0.703968,0.626984,0.706349,0.681111,0.032613,187
1,2.463731,0.034002,0.145152,0.010937,1,l2,word,1000,"(1, 3)","{'linearsvc__C': 1, 'linearsvc__penalty': 'l2'...",0.646825,0.659524,0.515873,0.637302,0.659524,0.623810,0.054613,299
2,5.083001,0.029989,0.256598,0.006509,1,l2,word,1000,"(1, 5)","{'linearsvc__C': 1, 'linearsvc__penalty': 'l2'...",0.646825,0.659524,0.515873,0.637302,0.659524,0.623810,0.054613,299
3,0.301410,0.012273,0.045825,0.001982,1,l2,word,5000,"(1, 1)","{'linearsvc__C': 1, 'linearsvc__penalty': 'l2'...",0.706746,0.716667,0.609524,0.626984,0.811905,0.694365,0.072394,173
4,2.401519,0.025374,0.145544,0.006093,1,l2,word,5000,"(1, 3)","{'linearsvc__C': 1, 'linearsvc__penalty': 'l2'...",0.706746,0.817857,0.527778,0.637302,0.811905,0.700317,0.109590,169
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
895,2.596360,0.023152,0.000000,0.000000,10,l1,char_wb,15000,"(1, 3)","{'linearsvc__C': 10, 'linearsvc__penalty': 'l1...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,458
896,4.748061,0.059630,0.000000,0.000000,10,l1,char_wb,15000,"(1, 5)","{'linearsvc__C': 10, 'linearsvc__penalty': 'l1...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,457
897,0.862964,0.042816,0.000000,0.000000,10,l1,char_wb,20000,"(1, 1)","{'linearsvc__C': 10, 'linearsvc__penalty': 'l1...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,456
898,2.596670,0.018641,0.000000,0.000000,10,l1,char_wb,20000,"(1, 3)","{'linearsvc__C': 10, 'linearsvc__penalty': 'l1...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,777


In [0]:
svm_results.to_csv('/content/gdrive/My Drive/smv_tuning1.csv')

# 2. Möglichkeit: Gridsearch auf allen Daten

In [0]:
gridsearch_full_svm = gridsearch = GridSearchCV(
    pipe_svm,
    pipe_svm_params,
    scoring='f1_macro',
    verbose=1,
    n_jobs=-1)

In [0]:
gridsearch_full_svm.fit(df.text, df.genre.to_numpy())

In [0]:
gridsearch_full_svm.best_params_, gridsearch_full_svm.best_score_

In [0]:
svm_full_results = pd.DataFrame.from_dict(gridsearch_full_svm.cv_results_)
svm_full_results

# 3. Möglichkeit: Alle Parametern mit normalen Durchläufen testen.

In [0]:
from sklearn.model_selection import ParameterGrid

In [19]:
df_train = df[df['period'] == 'P1']
test_dfs = [df[df['period'] == p] for p in df.period.unique() if p != 'P1']

15     P2
16     P2
17     P2
18     P2
19     P2
       ..
316    P2
317    P2
318    P2
319    P2
320    P2
Name: period, Length: 90, dtype: object

In [38]:
pipe_svm = Pipeline([('tfidf', tfidf), ('linearsvc', LinearSVC(loss='hinge'))])

pipe_svm_params = {
    'tfidf__max_features': [300, 1000, 5000,  10000, 20000],
    'tfidf__ngram_range': [(1,1), (1,3), (1,5)],
    'tfidf__lowercase': [True, False],
    'tfidf__stop_words': [None, get_stop_words('de')],
    'linearsvc__C': [0.1, 0.5, 1, 3, 7, 10],
    'linearsvc__penalty': ['l2', 'l1']
    
}
pipe_svm

Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=300,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('linearsvc',
                 LinearSVC(C=1.0, class_weight=None, dual=True,
                           fit_intercept=True, intercept_scaling=1,
          

In [0]:
pipe_svm_paramdict = ParameterGrid(pipe_svm_params)

In [0]:
from tqdm import tqdm_notebook
from sklearn.metrics import f1_score
import numpy as np
from pprint import pprint
import pickle


results = []
for current_params in tqdm_notebook(pipe_svm_paramdict):
  try:
    pipe_svm.set_params(**current_params)
    pipe_svm.fit(df_train.lemmas, df_train.genre)

    scores = []
    for df_test in test_dfs:
      y_pred = pipe_svm.predict(df_test.lemmas)
      scores.append(f1_score(df_test.genre, y_pred, average='macro'))
    result = {'avg_score': np.mean(scores), 'params': current_params}
    results.append(result)
    pprint(result['avg_score'])
    print()
  
  except Exception as e:
    result = {'avg_score': 0.0, 'params': current_params}
    results.append(result)
    print(f'Error on {current_params}')
  
  finally:
    # save results and go on 
    with open('svm_tuning1.h', 'wb') as f:
      pickle.dump([pipe_svm_params, results], f)



  

0.5680636408496424

0.5878308086474031

0.5233511869848431

0.5717065989358187

0.5233511869848431

0.5717065989358187

0.6248145137312628

0.688906675967685

0.5811398435541506

0.6984469264004635

0.5763488784778577

0.7027105163734709

0.39862833301752787

0.6917670408201323

0.6051776668110405

0.6956402749724699

0.6043856350191976

0.6995809735530766

0.40595857252179046

0.6937580456428256

0.39731868531292747

0.6873514111960566



In [0]:
import pickle

with open('svm_tuning1.h', 'wb') as f:
  pickle.dump([pipe_svm_params, results], f)